In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

**Repository Version** 
This notebook was run at commit:
``

# Input-Output Test
Previous input-output tests were done with the old DSelector event selection, so we need to repeat the I/O study with our new FSRoot-based event selection. We'll also reduce the binning to 10 MeV to match that of data, but we'll need to integrate over the entire $0.1\leq -t \leq 1.0$ range due to the lack of statistics for this signal MC dataset. This is safe to do, as there is no t-dependent physics generated in the dataset aside from an exponential curve, so binning finer in $-t$ only changes the statistics. The signal MC was generated with the following waveset:
* $b_1(1235)$ and $\rho(1450)$ Breit-Wigners fixed to their PDG values
  * an isotropic background is included, but is so small it's negligible
* No `OmegaDalitz` amplitudes for changing the dalitz distribution of the $\omega$ and its corresponding $\lambda$ distribution
* No $D/S$ ratio that would be typically associated with the $b_1$
* Only in the PARA 0 orientation

We mimic the waveset here, but will be doing a mass-independent fit, so no Breit-Wigner terms are used. The goal of this notebook is to see if any of our fits, after having passed through detector simulation and event selection, significantly fail to find the true values we generated. See the associated analysis note for details on how we extract the truth information from the generated data to compare to for our mass-independent fits. 

Most bins were performed with 100 randomized fits and 100 bootstrap fits. Some bins above 1.5 GeV required more than 100 randomized fits to have a successfully converged one. Two bins currently unfortunately fail to converge even with 5000 randomized fit attempts, though they are significantly above the generated amplitudes such that difficulties measuring them is to be expected. Fits are performed in 10 MeV mass-independent bins.

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager
import neutralb1.analysis.statistics as stats

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/io-tests/thin-bins/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
%%bash
# print out truth YAML file used to submit truth fits
cat $STUDY_DIR/truth_submission.YAML

In [ ]:
# load in preprocessed results
with open(f"{STUDY_DIR}/t_0.1-1.0/preprocessed_results.pkl", "rb") as f:
    data = pkl.load(f)
    results = ResultManager(**data)

In [ ]:
results.summary()

## Analysis

### Standard Plots
Lets view the standard set of plots to view how our model performed overall

In [ ]:
results.plot.intensity.jp()

In [ ]:
results.plot.intensity.waves()

In [ ]:
results.plot.intensity.waves(fractional=True)

In [ ]:
results.plot.diagnostic.matrix()

In [ ]:
results.plot.intensity.moments()

In [ ]:
# lets also make a quick plot of just the truth lines of the JP plot, for reference

Lets run a couple easy statistical tests for all fit indices

In [ ]:
sig_amplitudes = results.get_significant_amplitudes()
significant_phases = results.get_significant_phases()
columns = list(sig_amplitudes) + list(significant_phases)

if results.bootstrap_df is None: # assure type hinter that bootstrap df exists
    raise ValueError("Bootstrap dataframe is not available in results.")

stats.normality_test(results.fit_df, results.bootstrap_df, columns, alpha=0.01)

In [ ]:
if results.bootstrap_df is None: # assure type hinter that bootstrap df exists
    raise ValueError("Bootstrap dataframe is not available in results.")
stats.bias_test(results.fit_df, results.bootstrap_df, columns)

### Broad Exploration of Problematic Region
It's clear we're having issues in the 1.0-1.3 GeV mass region. Let's explore that on a wider scale to hopefully hone in on what could be the problem. First we'll look for any consistently strong correlations in this region

In [ ]:
if results.bootstrap_df is None: # assure type hinter that bootstrap df exists
    raise ValueError("Bootstrap dataframe is not available in results.")

fit_indices = list(np.arange(0, 31, dtype=int))

all_phases = results.phase_differences
significant_phases = results.get_significant_phases(fit_indices=fit_indices)
drop_phases = all_phases - significant_phases
columns_to_drop = list(drop_phases)
columns_to_drop.extend(["m", "p", "1p", "1m", "m1p", "p1p"])

stats.report_correlations(
    results.bootstrap_df, 
    fit_indices=fit_indices, 
    report_average=True, 
    drop_columns=columns_to_drop
)

In [ ]:
# TODO: make a jp style plot of just the truth JP values, to replace in analysis note